In [ ]:
# プリクラ完成品

import cv2
import dlib
import numpy as np
from imutils import face_utils
from scipy.spatial import distance
import time
import os

# Dlibの学習済みモデルファイルを読み込み
face_parts_detector = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
detector = dlib.get_frontal_face_detector()

# クマの左耳と右耳、鼻の画像を読み込み
left_bear_ear = cv2.imread("bear/left_bear_ear.png", cv2.IMREAD_UNCHANGED)
right_bear_ear = cv2.imread("bear/right_bear_ear.png", cv2.IMREAD_UNCHANGED)
bear_nose = cv2.imread("bear/bear_nose.png", cv2.IMREAD_UNCHANGED)

# クッキーの画像を読み込み
cookie_img = cv2.imread("bear/cookie.png", cv2.IMREAD_UNCHANGED)  # 透過PNG形式

# カウントダウン秒数
COUNTDOWN_SECONDS = 5

# 画像保存先のディレクトリ
save_dir = 'images'

# ディレクトリが存在しない場合は作成
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 透過PNG画像の合成関数
def overlay_image_alpha(img, overlay, pos, scale=1.0):
    overlay = cv2.resize(overlay, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
    x, y = pos
    h, w = overlay.shape[0], overlay.shape[1]

    # 境界チェック（オーバーレイ画像が元の画像をはみ出ないようにする）
    if x + w > img.shape[1]:
        w = img.shape[1] - x
    if y + h > img.shape[0]:
        h = img.shape[0] - y
    if x < 0:
        x = 0
    if y < 0:
        y = 0

    if overlay.shape[2] == 4:  # 透過チャンネルがある場合
        alpha_mask = overlay[:h, :w, 3] / 255.0
        alpha_image = 1.0 - alpha_mask

        # オーバーレイ部分を合成
        for c in range(0, 3):
            img[y:y+h, x:x+w, c] = (alpha_image * img[y:y+h, x:x+w, c] + alpha_mask * overlay[:h, :w, c])

    return img

# 口の開閉を判定する関数
def calc_mouth(mouth):
    A = distance.euclidean(mouth[2], mouth[10])
    B = distance.euclidean(mouth[4], mouth[8])
    C = distance.euclidean(mouth[0], mouth[6])
    MAR = (A + B) / (2.0 * C)
    return round(MAR, 3)

# カウントダウンを表示しながらフレームを更新する関数
def start_countdown(cap, detector, face_parts_detector):
    start_time = time.time()  # カウントダウンの開始時間
    while True:
        ret, frame = cap.read()  # フレームを再取得
        if not ret:
            break

        # カウントダウンタイマー計算
        elapsed_time = time.time() - start_time
        countdown_time = COUNTDOWN_SECONDS - int(elapsed_time)

        # カウントダウンが終了したらループを抜ける
        if countdown_time <= 0:
            break

        # グレースケールに変換
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # 顔検出
        rects = detector(gray, 1)

        for rect in rects:
            # ランドマークを取得
            shape = face_parts_detector(gray, rect)
            shape = face_utils.shape_to_np(shape)

            # クマさんフィルターを適用
            frame = apply_bear_filter(frame, shape)

        # カウントダウンの数字を表示
        cv2.putText(frame, str(countdown_time), (frame.shape[1] // 2 - 50, frame.shape[0] // 2), 
                    cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 5, cv2.LINE_AA)
        
        # フレームを表示
        cv2.imshow('Preview', frame)

        # 'q'キーでカウントダウン中断
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    return frame  # カウントダウン後の最後のフレームを返す

# クマさんフィルターを適用する関数
def apply_bear_filter(image, shape):
    left_eye = shape[36]
    right_eye = shape[45]
    nose = shape[30]
    chin = shape[8]

    face_width = np.linalg.norm(left_eye - right_eye)
    face_height = np.linalg.norm(nose - chin)

    # 左耳と右耳の位置計算
    left_ear_x = int(left_eye[0]) - int(face_width * 0.5)
    left_ear_y = int(left_eye[1]) - int(face_height * 1.2)
    right_ear_x = int(right_eye[0]) + int(face_width * -0.2)
    right_ear_y = int(right_eye[1]) - int(face_height * 1.25)

    # クマの耳と鼻を合成
    image = overlay_image_alpha(image, left_bear_ear, (left_ear_x, left_ear_y), scale=face_width / 200)
    image = overlay_image_alpha(image, right_bear_ear, (right_ear_x, right_ear_y), scale=face_width / 200)

    # クマの鼻を合成
    nose_x = nose[0] - 58
    nose_y = nose[1] - 70
    image = overlay_image_alpha(image, bear_nose, (nose_x, nose_y), scale=0.3)

    # 口のランドマークを取得し、口が開いているか確認
    mouth = shape[48:68]
    MAR = calc_mouth(mouth)
    if MAR > 0.6:
        mouth_center = mouth.mean(axis=0).astype(int)
        lip_distance = distance.euclidean(mouth[3], mouth[9])
        image = overlay_image_alpha(image, cookie_img, (mouth_center[0] - 20, mouth_center[1] - int(lip_distance * 0.5)), scale=0.5)

    return image

# 画像を保存する関数
def save_image(frame):
    filename = f'photo_{time.strftime("%Y%m%d_%H%M%S")}.png'
    filepath = os.path.join(save_dir, filename)
    cv2.imwrite(filepath, frame)
    print(f"画像が保存されました: {filepath}")

# カメラデバイスをオープン
cap = cv2.VideoCapture(0)

###################
# メイン処理       #
###################
while True:
    ret, image = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)

    for rect in rects:
        shape = face_parts_detector(gray, rect)
        shape = face_utils.shape_to_np(shape)

        # クマさんフィルターをリアルタイムで適用
        image = apply_bear_filter(image, shape)

    # フレームを表示
    cv2.imshow('Preview', image)

    # 's'キーでカウントダウン開始
    if cv2.waitKey(1) & 0xFF == ord('s'):
        # カウントダウンを開始し、終了時のフレームを取得
        final_frame = start_countdown(cap, detector, face_parts_detector)

        # カウントダウン後のフレームにクマさんフィルターを適用
        gray = cv2.cvtColor(final_frame, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 1)

        for rect in rects:
            shape = face_parts_detector(gray, rect)
            shape = face_utils.shape_to_np(shape)

            # カウントダウン後のフレームにクマさんフィルターを適用
            final_frame = apply_bear_filter(final_frame, shape)

        # フィルター適用済みの画像を保存
        save_image(final_frame)

    # 'q'キーで終了
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [7]:
import cv2
import dlib
import numpy as np
from imutils import face_utils
from scipy.spatial import distance
import time
import os
import pygame  # 追加

# pygameの初期化
pygame.mixer.init()

# シャッター音のファイルパスを設定
shutter_sound_path = "sounds/shutter.mp3"

# Dlibの学習済みモデルファイルを読み込み
face_parts_detector = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
detector = dlib.get_frontal_face_detector()

# クマの左耳と右耳、鼻の画像を読み込み
left_bear_ear = cv2.imread("bear/left_bear_ear.png", cv2.IMREAD_UNCHANGED)
right_bear_ear = cv2.imread("bear/right_bear_ear.png", cv2.IMREAD_UNCHANGED)
bear_nose = cv2.imread("bear/bear_nose.png", cv2.IMREAD_UNCHANGED)

# クッキーの画像を読み込み
cookie_img = cv2.imread("bear/cookie.png", cv2.IMREAD_UNCHANGED)  # 透過PNG形式

# カウントダウン秒数
COUNTDOWN_SECONDS = 5

# 画像保存先のディレクトリ
save_dir = 'images'

# ディレクトリが存在しない場合は作成
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 透過PNG画像の合成関数
def overlay_image_alpha(img, overlay, pos, scale=1.0):
    overlay = cv2.resize(overlay, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
    x, y = pos
    h, w = overlay.shape[0], overlay.shape[1]

    # 境界チェック（オーバーレイ画像が元の画像をはみ出ないようにする）
    if x + w > img.shape[1]:
        w = img.shape[1] - x
    if y + h > img.shape[0]:
        h = img.shape[0] - y
    if x < 0:
        x = 0
    if y < 0:
        y = 0

    if overlay.shape[2] == 4:  # 透過チャンネルがある場合
        alpha_mask = overlay[:h, :w, 3] / 255.0
        alpha_image = 1.0 - alpha_mask

        # オーバーレイ部分を合成
        for c in range(0, 3):
            img[y:y+h, x:x+w, c] = (alpha_image * img[y:y+h, x:x+w, c] + alpha_mask * overlay[:h, :w, c])

    return img

# 口の開閉を判定する関数
def calc_mouth(mouth):
    A = distance.euclidean(mouth[2], mouth[10])
    B = distance.euclidean(mouth[4], mouth[8])
    C = distance.euclidean(mouth[0], mouth[6])
    MAR = (A + B) / (2.0 * C)
    return round(MAR, 3)

# カウントダウンを表示しながらフレームを更新する関数
def start_countdown(cap, detector, face_parts_detector):
    start_time = time.time()  # カウントダウンの開始時間
    while True:
        ret, frame = cap.read()  # フレームを再取得
        if not ret:
            break

        # カウントダウンタイマー計算
        elapsed_time = time.time() - start_time
        countdown_time = COUNTDOWN_SECONDS - int(elapsed_time)

        # カウントダウンが終了したらループを抜ける
        if countdown_time <= 0:
            break

        # グレースケールに変換
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # 顔検出
        rects = detector(gray, 1)

        for rect in rects:
            # ランドマークを取得
            shape = face_parts_detector(gray, rect)
            shape = face_utils.shape_to_np(shape)

            # クマさんフィルターを適用
            frame = apply_bear_filter(frame, shape)

        # カウントダウンの数字を表示
        cv2.putText(frame, str(countdown_time), (frame.shape[1] // 2 - 50, frame.shape[0] // 2), 
                    cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 5, cv2.LINE_AA)
        
        # フレームを表示
        cv2.imshow('Preview', frame)

        # 'q'キーでカウントダウン中断
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    return frame  # カウントダウン後の最後のフレームを返す

# クマさんフィルターを適用する関数
def apply_bear_filter(image, shape):
    left_eye = shape[36]
    right_eye = shape[45]
    nose = shape[30]
    chin = shape[8]

    face_width = np.linalg.norm(left_eye - right_eye)
    face_height = np.linalg.norm(nose - chin)

    # 左耳と右耳の位置計算
    left_ear_x = int(left_eye[0]) - int(face_width * 0.5)
    left_ear_y = int(left_eye[1]) - int(face_height * 1.2)
    right_ear_x = int(right_eye[0]) + int(face_width * -0.2)
    right_ear_y = int(right_eye[1]) - int(face_height * 1.25)

    # クマの耳と鼻を合成
    image = overlay_image_alpha(image, left_bear_ear, (left_ear_x, left_ear_y), scale=face_width / 200)
    image = overlay_image_alpha(image, right_bear_ear, (right_ear_x, right_ear_y), scale=face_width / 200)

    # クマの鼻を合成
    nose_x = nose[0] - 58
    nose_y = nose[1] - 70
    image = overlay_image_alpha(image, bear_nose, (nose_x, nose_y), scale=0.3)

    # 口のランドマークを取得し、口が開いているか確認
    mouth = shape[48:68]
    MAR = calc_mouth(mouth)
    if MAR > 0.6:
        mouth_center = mouth.mean(axis=0).astype(int)
        lip_distance = distance.euclidean(mouth[3], mouth[9])
        image = overlay_image_alpha(image, cookie_img, (mouth_center[0] - 20, mouth_center[1] - int(lip_distance * 0.5)), scale=0.5)

    return image

# 画像を保存する関数
def save_image(frame):
    filename = f'photo_{time.strftime("%Y%m%d_%H%M%S")}.png'
    filepath = os.path.join(save_dir, filename)
    cv2.imwrite(filepath, frame)
    print(f"画像が保存されました: {filepath}")

# シャッター音を再生する関数
def play_shutter_sound():
    pygame.mixer.music.load(shutter_sound_path)
    pygame.mixer.music.play()

# カメラデバイスをオープン
cap = cv2.VideoCapture(0)

###################
# メイン処理       #
###################
while True:
    ret, image = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)

    for rect in rects:
        shape = face_parts_detector(gray, rect)
        shape = face_utils.shape_to_np(shape)

        # クマさんフィルターをリアルタイムで適用
        image = apply_bear_filter(image, shape)

    # フレームを表示
    cv2.imshow('Preview', image)

    # 's'キーでカウントダウン開始
    if cv2.waitKey(1) & 0xFF == ord('s'):
        # カウントダウンを開始し、終了時のフレームを取得
        final_frame = start_countdown(cap, detector, face_parts_detector)

        # シャッター音を再生
        play_shutter_sound()

        # カウントダウン後のフレームにクマさんフィルターを適用
        gray = cv2.cvtColor(final_frame, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 1)

        for rect in rects:
            shape = face_parts_detector(gray, rect)
            shape = face_utils.shape_to_np(shape)

            # カウントダウン後のフレームにクマさんフィルターを適用
            final_frame = apply_bear_filter(final_frame, shape)

        # フィルター適用済みの画像を保存
        save_image(final_frame)

    # 'q'キーで終了
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
画像が保存されました: images\photo_20241003_162208.png
